# Interactive generation of graphene moiré lattices
Author: [Tobias A. de Jong](https://github.com/TAdeJong)

This notebook uses `numpy`, `dask` and `matplotlib` to easily generate moiré lattices of two hexagonal graphene lattices. Using `ipywidgets`, we create an interactive plot where the angle can be adopted.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools as itert

import ipywidgets as widgets

import dask.array as da
from dask.distributed import Client, LocalCluster

import colorcet as cc

In [2]:
# Create a dask cluster for parallel computation. Settings optimized to play nice with mybinder.org
cluster = LocalCluster(n_workers=1, threads_per_worker=4, memory_limit='2GB')  
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:41529 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


In [3]:
def rotate(vec, angle):
    "Rotate `vec` by `angle` around the origin"
    R = np.array([[np.cos(angle), -np.sin(angle)], 
                  [np.sin(angle), np.cos(angle)]])
    return R @ vec

In [4]:
S = 250
r_k = 0.2 
t0 = np.deg2rad(15)
k0 = r_k*np.array([np.cos(t0), np.sin(t0)])
a_0 = 0.246  #Lattice constant of graphene in nm

In [5]:
def lattice_gen(k0, theta, order, size=250):
    """Generate a regular hexagonal lattice of `2*size` times `2*size`.
    The lattice is generated from the six 60 degree rotations of `k0`,
    further rotated by `theta` degrees. With higher order frequency
    components upto order `order`
    
    The generated lattice gets returned as a dask array.
    """
    xx, yy = da.meshgrid(np.arange(-size,size), np.arange(-size,size), indexing='ij')
    ks = np.stack([rotate(rotate(k0, np.deg2rad(theta)), np.pi/3*i) for i in range(3)])
    ks = np.concatenate([ks, -ks, np.array([[0,0]])])
    # Yes, this is probably not the smartest way to do this
    tks = np.array([np.sum(ksp, axis=0) for ksp in itert.product(*[ks]*order)])
    rks, k_c = np.unique(tks, return_counts=True, axis=0)
    rks = da.from_array(rks, chunks=(13,2))
    iterated = k_c[:,None,None]*np.exp(np.pi*2*1j*(xx*rks[:,0,None,None] + yy*rks[:,1,None,None]))
    iterated = iterated.sum(axis=0)
    # Now add the second shifted sublattice lattice to get a hexagonal lattice
    x = np.array([ks[1], -ks[2]])
    shift = (np.linalg.inv(x).T/3).sum(axis=0)  # Don't ask, this works
    iterated += (k_c[:,None,None]*np.exp(np.pi*2*1j*((xx + shift[0])*rks[:,0,None,None] + 
                                             (yy + shift[1])*rks[:,1,None,None]))).sum(axis=0)
    return iterated.real


In [6]:
iterated1 = 0.7*lattice_gen(k0, 0, 2, S).persist()

def plot(theta=1.05):
    "Make a plot of a second lattice rotated by `theta` degrees on top of `iterated1`"
    fig, ax = plt.subplots(figsize=[10,10])
    iterated2 = lattice_gen(k0, theta, 2, S)
    data = (iterated1 + iterated2).compute()
    im = ax.imshow(data.T, cmap='cet_fire_r', extent=[-S*r_k*a_0, S*r_k*a_0, S*r_k*a_0, -S*r_k*a_0], 
                   vmax=np.quantile(data,0.9),
                   vmin=np.quantile(data,0.1),
                  )
    ax.set_xlabel('x (nm)')
    ax.set_ylabel('y (nm)')
    ax.set_title(f'$\\theta = {theta:.2f}^\\circ$')

widgets.interactive(plot, 
    theta=widgets.FloatSlider(value=1.05, min=0, max=5,step=0.05, continuous_update=False),
   )

interactive(children=(FloatSlider(value=1.05, continuous_update=False, description='theta', max=5.0, step=0.05…